In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from scipy import interp
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

### Load the data

In [17]:
#open the locally saved csv
df = pd.read_csv('C:/Users/595217/general-assembly/project-final/data/mgm.csv', usecols=['description', 'jobtype', 'usetype'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17669 entries, 0 to 17668
Data columns (total 3 columns):
description    17669 non-null object
jobtype        17669 non-null object
usetype        17669 non-null object
dtypes: object(3)
memory usage: 414.2+ KB


### Create the features and target variables

In [19]:
df = df[df.usetype != 'Mixed Occupancy']

In [20]:
X = df['description']
z = df['usetype']
y = df['jobtype']

In [21]:
# y = y.map({'Commercial': 0, 'Residential': 1})
y = y.map({'New': 0, 'Existing': 1, 'Alteration': 2, 'Repair': 3, 'Other': 4, 'Addition': 5})

In [51]:
y.value_counts(), z.value_counts()

(0    4820
 1    4017
 2    3083
 3    2322
 4    2264
 5    1143
 Name: jobtype, dtype: int64, Residential    10439
 Commercial      7210
 Name: usetype, dtype: int64)

In [23]:
# y_bin = label_binarize(y, classes=[0,1,2])
# n_y_bin_classes = y_bin.shape[1]
z_bin = label_binarize(z, classes=[0,1,2,3,4,5])
n_z_bin_classes = z_bin.shape[1]

In [24]:
X_tr, X_te, y_tr, y_te, = train_test_split(X, y, test_size=0.30, random_state=42)

In [25]:
# create the BOW representation
bow_transform = text.CountVectorizer(min_df=0, stop_words="english")
X_tr_bow = bow_transform.fit_transform(X_tr)
X_te_bow = bow_transform.transform(X_te)
len(bow_transform.vocabulary_)

8190

In [26]:
#create tf-idf representation using the bow matrix
tfidf_trfm = text.TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_trfm.fit_transform(X_tr_bow)
X_te_tfidf = tfidf_trfm.transform(X_te_bow)
X_te_tfidf.shape

(5295, 8190)

### Instantiate a random forest classifier, run on the target variables and score the model

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [27]:
#instantiate the classifier
rf = OneVsRestClassifier(RandomForestClassifier())

In [28]:
rf.fit(X_tr_tfidf, y_tr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
y_pred = rf.predict(X_te_tfidf)
print(rf.__class__.__name__, accuracy_score(y_te, y_pred))

RandomForestClassifier 0.9242681775259679


In [30]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("StDev:", scores.std())

In [31]:
y_tr_tfidf_scores = cross_val_score(rf, X_tr_tfidf, y_tr, cv=10, scoring="accuracy")

In [32]:
display_scores(y_tr_tfidf_scores)

Scores: [0.90153349 0.9135004  0.90703314 0.90850202 0.91255061 0.91093117
 0.90923825 0.91247974 0.91247974 0.89384117]
Mean: 0.9082089747223531
StDev: 0.005867384824918001


In [33]:
y_te_tfidf_scores = cross_val_score(rf, X_te_tfidf, y_te, cv=10, scoring="accuracy")

In [34]:
display_scores(y_te_tfidf_scores)

Scores: [0.85928705 0.82330827 0.86037736 0.85849057 0.82986767 0.83175803
 0.84877127 0.86174242 0.84469697 0.81024668]
Mean: 0.8428546298295612
StDev: 0.017186729511305333


In [35]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

C:\Users\595217\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [36]:
model = Sequential()

In [37]:
model.add(Dense(12, input_dim=8190, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, input_dim=8190, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, input_dim=8190, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, input_dim=8190, kernel_initializer='uniform', activation='sigmoid'))

In [38]:
model.output_shape

(None, 1)

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                98292     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 98,477
Trainable params: 98,477
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
model.fit(X_tr_tfidf, y_tr, batch_size=32, epochs=100, verbose=1, validation_data=(X_te_tfidf, y_te))

Train on 12354 samples, validate on 5295 samples
Epoch 1/100
12354/12354 [==============================] - 3s 275us/step - loss: -10.1633 - acc: 0.3555 - val_loss: -15.7314 - val_acc: 0.4465
Epoch 2/100
12354/12354 [==============================] - 3s 230us/step - loss: -16.7011 - acc: 0.4424 - val_loss: -16.0067 - val_acc: 0.4616
Epoch 3/100
12354/12354 [==============================] - 3s 229us/step - loss: -16.9430 - acc: 0.4555 - val_loss: -16.1285 - val_acc: 0.4752
Epoch 4/100
12354/12354 [==============================] - 3s 228us/step - loss: -17.0443 - acc: 0.4645 - val_loss: -16.1777 - val_acc: 0.4795
Epoch 5/100
12354/12354 [==============================] - 3s 229us/step - loss: -17.0888 - acc: 0.4679 - val_loss: -16.2015 - val_acc: 0.4799
Epoch 6/100
12354/12354 [==============================] - 4s 285us/step - loss: -17.1171 - acc: 0.4698 - val_loss: -16.2227 - val_acc: 0.4856
Epoch 7/100
12354/12354 [==============================] - 3s 241us/step - loss: -17.1406 - a

12354/12354 [==============================] - 3s 258us/step - loss: -17.2076 - acc: 0.4777 - val_loss: -16.1729 - val_acc: 0.4886
Epoch 58/100
12354/12354 [==============================] - 3s 225us/step - loss: -17.2080 - acc: 0.4778 - val_loss: -16.1715 - val_acc: 0.4886
Epoch 59/100
12354/12354 [==============================] - 3s 225us/step - loss: -17.2079 - acc: 0.4778 - val_loss: -16.1702 - val_acc: 0.4888
Epoch 60/100
12354/12354 [==============================] - 3s 225us/step - loss: -17.2081 - acc: 0.4778 - val_loss: -16.1673 - val_acc: 0.4890
Epoch 61/100
12354/12354 [==============================] - 3s 224us/step - loss: -17.2077 - acc: 0.4778 - val_loss: -16.1795 - val_acc: 0.4891
Epoch 62/100
12354/12354 [==============================] - 3s 269us/step - loss: -17.2044 - acc: 0.4773 - val_loss: -16.1607 - val_acc: 0.4886
Epoch 63/100
12354/12354 [==============================] - 3s 237us/step - loss: -17.2092 - acc: 0.4779 - val_loss: -16.1710 - val_acc: 0.4890
Epoch

In [42]:
score = model.evaluate(X_te_tfidf, y_te, batch_size=32)

5295/5295 [==============================] - 1s 116us/step


In [43]:
y_pred = model.predict(X_te_tfidf, batch_size=32)

### Convert predictions to text labels and join with the main dataframe to see where the errors occurred 

In [44]:
y_pred_classes = []
for i in y_pred:
    if i >= .5:
        y_pred_classes.append(1)
    else:
        y_pred_classes.append(0)

In [45]:
real_v_pred = pd.concat([y_te.reset_index(), pd.Series(y_pred_classes)], axis=1, join='inner', ignore_index=True).reset_index(drop=True).set_index([1])

In [46]:
real_v_pred.reset_index(inplace=True)
real_v_pred.columns = ['test', 'index', 'pred']

In [47]:
errors = real_v_pred[real_v_pred.test != real_v_pred.pred]
errors.set_index(keys='index', inplace=True)

In [48]:
df_errors = df.join(errors, how='inner')

In [49]:
df_errors.to_csv('df_errors.csv')

### Create a deep learning model with keras and compare to the random forest

In [50]:
from sklearn.metrics import roc_curve
y_pred_keras = model.predict(X_te_tfidf).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_te, y_pred_keras)

ValueError: multiclass format is not supported

In [ ]:
y_pred_keras = model.predict(X_te_tfidf, batch_size=32).ravel()

In [ ]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_te, y_pred_keras)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
y_pred_rf = rf.predict_proba(X_te_tfidf)[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_te, y_pred_rf)
auc_rf = auc(fpr_rf, tpr_rf)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.98, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()